# PF-CZM Simulation of Crack Propagation in a Glass Plate with Notch
Colab-Compatible, No `mshr`, FEniCS 2019.1.0

SyntaxError: invalid syntax (1625531121.py, line 1)

In [1]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'fenics'

In [ ]:
# Parameters
length = 100.0
height = 40.0
notch_length = 50.0
notch_height = 20.0

E = 32e9
nu = 0.2
Gf = 3.8
ft = 12e6
l = 0.25
plane_stress = True

mesh_res_x = 200
mesh_res_y = 80

In [ ]:
# Mesh
mesh = RectangleMesh(Point(0.0, 0.0), Point(length, height), mesh_res_x, mesh_res_y)

In [ ]:
# Function spaces
V_u = VectorFunctionSpace(mesh, 'CG', 1)
V_d = FunctionSpace(mesh, 'CG', 1)

u = Function(V_u, name="Displacement")
d = Function(V_d, name="Damage")
v = TestFunction(V_u)
w = TestFunction(V_d)

In [ ]:
# Material model
mu = E / (2*(1 + nu))
lmbda = E*nu / ((1 + nu)*(1 - 2*nu)) if not plane_stress else E*nu / (1 - nu**2)

def sigma(u, d):
    eps = sym(grad(u))
    return (1 - d)**2 * (lmbda*tr(eps)*Identity(2) + 2.0*mu*eps)

Gc = Gf * 1e3

In [ ]:
# Boundary conditions
class TopBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], height) and on_boundary

class BottomBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0) and on_boundary

boundary_markers = MeshFunction("size_t", mesh, mesh.topology().dim() - 1, 0)
top = TopBoundary()
bottom = BottomBoundary()
top.mark(boundary_markers, 1)
bottom.mark(boundary_markers, 2)

bcs_u = [DirichletBC(V_u.sub(1), Constant(0.001), top),
         DirichletBC(V_u.sub(1), Constant(0.0), bottom)]
bcs_d = []

In [ ]:
# Variational forms
u_old = Function(V_u)
d_old = Function(V_d)

elastic_energy = 0.5 * inner(sigma(u, d), sym(grad(u))) * dx
fracture_energy = Gc * (d**2 / (2*l) + l/2 * dot(grad(d), grad(d))) * dx
total_energy = elastic_energy + fracture_energy

res_u = derivative(total_energy, u, v)
res_d = derivative(total_energy, d, w)

In [ ]:
# Solvers
problem_u = NonlinearVariationalProblem(res_u, u, bcs_u)
solver_u = NonlinearVariationalSolver(problem_u)

problem_d = NonlinearVariationalProblem(res_d, d, bcs_d)
solver_d = NonlinearVariationalSolver(problem_d)

In [ ]:
# Solve
for step in range(1):
    solver_u.solve()
    solver_d.solve()

In [ ]:
# Postprocess
plt.figure(figsize=(10, 2))
p = plot(d, title="Damage Field (Crack)", cmap="gray_r")
plt.colorbar(p)
plt.savefig("damage_field.png", dpi=300)
plt.show()

File("damage_field.pvd") << d